In [1]:
from pymgpipe import load_model, get_reactions, compute_multi_sample_nmpcs

model = load_model('C:/Users/yolim/Documents/Columbia/msefba_analysis/elovitz_ag1_combined_all.mps',solver='cplex')
# reactions = set([m.name.split('_mc')[0] for m in get_reactions(model,regex='^EX_.*_m_.*$')])
# compute_multi_sample_nmpcs(model,reactions[:10])



Selected objective sense:  MINIMIZE
Selected objective  name:  a3ba83a3-4042-11ed-8c17-2cf05ddff7a4
Selected bound      name:  bnd


In [ ]:
reactions = list(set([m.name.split('_mc')[0] for m in get_reactions(model,regex='^EX_.*_m_.*$')]))
compute_multi_sample_nmpcs(model,reactions[:10])

In [14]:
from pymgpipe import compute_multi_sample_nmpcs, solve_multi_sample_model
from pymgpipe.optlang_util import _get_reverse_id
import optlang
import numpy as np

metab = 'EX_tyr_L_m'
metab_rxns = get_reactions(model,regex='^%s_mc.*$'%metab)
net_rxns = [f-model.variables[_get_reverse_id(f.name,multi_sample=True)] for f in metab_rxns]
s_obj = np.sum(net_rxns)

model.objective = optlang.Objective(s_obj,direction='min')
min_sol = solve_multi_sample_model(model,verbosity=3,method='auto',reactions=[m.name for m in metab_rxns])


Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 12 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 10 threads...
 * Starting primal Simplex on 1 thread...

Dual simplex solved model.

Problem solved!


In [15]:
min_sol

,mc1,mc10,mc100,mc101,mc102,mc103,mc104,mc105,mc106,mc107,...,mc90,mc91,mc92,mc93,mc94,mc95,mc96,mc97,mc98,mc99
EX_tyr_L_m,-216.844332,-19.251925,-0.133177,-88.364007,-265.62655,-4.07832,-126.986513,-2.192957,-222.080981,-100.466046,...,-203.353373,-9.660848,-132.688315,-27.991863,-41.849386,-1.481664,-1.080271,-53.247958,-123.477889,-187.658641


In [11]:
fluxes={}
rxns = get_reactions(model,reactions=[m.name for m in metab_rxns])
for forward in rxns:
    sample_id = 'mc'+forward.name.split('_mc')[1]
    r_id = _get_reverse_id(forward.name, multi_sample=True)
    if r_id not in model.variables:
        continue
    reverse = model.variables[r_id]

    flux = float(forward.primal-reverse.primal)
    flux = 0 if flux == -0.0 else flux
    flux = flux if abs(flux)>1e-5 else 0
    if sample_id not in fluxes:
        fluxes[sample_id] = {}
    fluxes[sample_id][forward.name.split('_mc')[0]]=flux
fluxes

{'mc1': {'EX_tyr_L_m': -216.8443320891836},
 'mc10': {'EX_tyr_L_m': -19.251924925503474},
 'mc100': {'EX_tyr_L_m': -0.13317679097641003},
 'mc101': {'EX_tyr_L_m': -88.36400731445754},
 'mc102': {'EX_tyr_L_m': -265.6265496205826},
 'mc103': {'EX_tyr_L_m': -4.078320389112744},
 'mc104': {'EX_tyr_L_m': -126.98651338759713},
 'mc105': {'EX_tyr_L_m': -2.192956925337395},
 'mc106': {'EX_tyr_L_m': -222.08098140851152},
 'mc107': {'EX_tyr_L_m': -100.46604560092992},
 'mc108': {'EX_tyr_L_m': -167.4376538785372},
 'mc109': {'EX_tyr_L_m': -127.53658346614094},
 'mc11': {'EX_tyr_L_m': -0.5467468290709576},
 'mc110': {'EX_tyr_L_m': -6.830830100054024},
 'mc111': {'EX_tyr_L_m': -56.00214688278405},
 'mc112': {'EX_tyr_L_m': -208.2244691848713},
 'mc113': {'EX_tyr_L_m': -88.53149880849884},
 'mc114': {'EX_tyr_L_m': -166.02782613891281},
 'mc115': {'EX_tyr_L_m': -66.10890337328189},
 'mc116': {'EX_tyr_L_m': -1.7124324250014042},
 'mc117': {'EX_tyr_L_m': -4.576041125457699},
 'mc118': {'EX_tyr_L_m': -71